# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>
<hr>


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that a too large number of sentences will make your compute slow down, or even freeze - your RAM can even overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [1]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2022-02-03 20:01:14.621131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-03 20:01:14.621275: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


2022-02-03 20:01:23.453407: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2022-02-03 20:01:23.453517: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-KQUNRQ54): /proc/driver/nvidia/version does not exist
2022-02-03 20:01:23.455305: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure : 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [2]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train, vector_size = 100, min_count = 10, window = 6)

In [5]:
word2vec.wv[0]

array([ 0.87986517, -0.18382567,  0.2816751 ,  0.03649974, -0.7686907 ,
       -0.36882693, -0.14136596,  0.81198746, -0.9606589 , -0.624932  ,
        0.20872988, -0.34505993,  0.12400606,  0.77152544,  0.06400364,
       -0.0236338 ,  0.23534687,  0.34574562, -0.50934726, -1.9771593 ,
        0.03436977,  0.596651  ,  0.672445  , -0.42333674,  0.02952801,
       -0.37855235,  0.63452584,  0.10470641, -0.66254854, -0.10995002,
       -0.292759  ,  0.57641935,  0.92199874, -1.2787434 ,  0.09404639,
        0.09261798,  0.83391565, -0.06339327, -0.15867162, -0.24388176,
       -0.5079075 ,  0.09965529, -0.8760894 , -0.05570967,  0.0304227 ,
       -0.78985333, -0.2766121 ,  1.0075377 , -0.05725787,  0.5955523 ,
       -0.6641223 ,  0.01684951, -1.113806  ,  0.74939984,  0.6178747 ,
        0.49316993, -0.55832076, -0.40789977, -0.22069435,  0.03694138,
        0.6372632 , -0.14248656, -0.07525415,  0.00683134, -0.28158292,
        0.09841053, -0.45956993, -0.28514543, -0.05068856,  0.05

Let's reuse the functions of the previous exercise to convert your training and test data into something you can fed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad` :
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from searching at errors too late and from letting them propagate through the entire notebook.

In [9]:
X_train_pad[0]

array([[ 0.89005506,  0.22253354,  0.7855564 , ..., -0.18248144,
        -0.18397295, -0.54403347],
       [ 0.44280338, -0.50045085,  0.6310196 , ..., -1.2205391 ,
         1.3660139 ,  1.0935569 ],
       [-0.0286577 ,  0.51957184,  0.25439858, ..., -0.41317296,
        -0.6410761 ,  0.41316113],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [4]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you do something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [8]:
baseline_acc = 1 - y_train.sum() / y_train.size #all reviews are negative
baseline_acc

0.506

# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [14]:
from tensorflow.keras import layers, Sequential


model = Sequential()
model.add(layers.Masking(mask_value=0))
model.add(layers.LSTM(20, activation="tanh"))
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))


❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy with greatly depend on your training test corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

In [17]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience = 4)

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics = ['acc'])

In [18]:
history = model.fit(X_train_pad, y_train,
         epochs = 50,
         batch_size = 128,
         verbose = 1,
         validation_split=0.3,
         callbacks=[es])

Epoch 1/50
14/14 [==============================] - 12s 401ms/step - loss: 0.6924 - acc: 0.5143 - val_loss: 0.6825 - val_acc: 0.5773
Epoch 2/50
14/14 [==============================] - 3s 250ms/step - loss: 0.6841 - acc: 0.5514 - val_loss: 0.6799 - val_acc: 0.5853
Epoch 3/50
14/14 [==============================] - 4s 258ms/step - loss: 0.6775 - acc: 0.5840 - val_loss: 0.6767 - val_acc: 0.5733
Epoch 4/50
14/14 [==============================] - 4s 279ms/step - loss: 0.6689 - acc: 0.5977 - val_loss: 0.6603 - val_acc: 0.6160
Epoch 5/50
14/14 [==============================] - 4s 259ms/step - loss: 0.6605 - acc: 0.6280 - val_loss: 0.6486 - val_acc: 0.6400
Epoch 6/50
14/14 [==============================] - 4s 258ms/step - loss: 0.6488 - acc: 0.6331 - val_loss: 0.6409 - val_acc: 0.6293
Epoch 7/50
14/14 [==============================] - 4s 255ms/step - loss: 0.6452 - acc: 0.6314 - val_loss: 0.6337 - val_acc: 0.6613
Epoch 8/50
14/14 [==============================] - 4s 258ms/step - loss: 0

❓ **Question** ❓ Evaluate your model on the test set

In [19]:
model.evaluate(X_test_pad, y_test)

79/79 [==============================] - 3s 43ms/step - loss: 0.6248 - acc: 0.6652


[0.6247703433036804, 0.6651999950408936]

# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. On the other hand, let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that other have learnt? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this : 

In [20]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ You can also find the list of the models and their size on the [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models).

❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

<details>
    <summary>💡 Hint</summary>
    
The `glove-wiki-gigaword-50` model is a good candidate to start with as it is the smaller (65 MB).

</details>

In [21]:
word2vec_transfer = api.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [31]:
len(word2vec_transfer.index_to_key)

400000

In [34]:
word2vec_transfer.vector_size

100

❓ **Question** ❓ Let's embedd `X_train` and `X_test`, as in the first question where we provided the functions to so ! (There is a slight different in the `embed_sentence_with_TF` function that we will not dig into)

In [35]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [36]:
# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training could take some time here. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [38]:
model_2 = Sequential()
model_2.add(layers.Masking(mask_value=0))
model_2.add(layers.LSTM(20, activation="tanh"))
model_2.add(layers.Dense(10, activation="relu"))
model_2.add(layers.Dense(1, activation="sigmoid"))

model_2.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics = ['acc'])

In [41]:
es = EarlyStopping(patience = 10)

history = model_2.fit(X_train_pad_2, y_train,
         epochs = 50,
         batch_size = 64,
         verbose = 1,
         validation_split=0.2,
         callbacks=[es])

Epoch 1/50
32/32 [==============================] - 8s 259ms/step - loss: 0.5554 - acc: 0.7235 - val_loss: 0.5552 - val_acc: 0.7100
Epoch 2/50
32/32 [==============================] - 8s 257ms/step - loss: 0.5619 - acc: 0.7240 - val_loss: 0.5764 - val_acc: 0.6960
Epoch 3/50
32/32 [==============================] - 8s 254ms/step - loss: 0.5333 - acc: 0.7385 - val_loss: 0.5348 - val_acc: 0.7540
Epoch 4/50
32/32 [==============================] - 8s 246ms/step - loss: 0.5316 - acc: 0.7320 - val_loss: 0.5901 - val_acc: 0.6840
Epoch 5/50
32/32 [==============================] - 9s 280ms/step - loss: 0.5252 - acc: 0.7485 - val_loss: 0.5239 - val_acc: 0.7360
Epoch 6/50
32/32 [==============================] - 8s 253ms/step - loss: 0.5195 - acc: 0.7495 - val_loss: 0.5392 - val_acc: 0.7140
Epoch 7/50
32/32 [==============================] - 9s 268ms/step - loss: 0.4946 - acc: 0.7670 - val_loss: 0.5328 - val_acc: 0.7240
Epoch 8/50
32/32 [==============================] - 9s 271ms/step - loss: 0.

In [43]:
res = model_2.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 70.880%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarder words that were not present more than a given number of time in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously